In [5]:
import json 
ola=json.load(open('ola.json'))

In [9]:
import pandas as pd
df=pd.DataFrame(ola).transpose()

In [10]:
df

,date,distance,amount,address
OLA_1.pdf,10-12-2018,165.0,2596.0,"[Tagore Park, EM Bypass, Naskarhat, Kasba, Kol..."
OLA_2.pdf,26-11-2014,27.2,415.0,[]
OLA_3.pdf,13-09-2017,172.97,2680.0,"[Road Number 23, Andheri East, Mumbai, Maharas..."
OLA_4.pdf,12-01-2016,23.95,416.0,[]
OLA_5.pdf,09-05-2016,14.63,284.0,[]
OLA_6.pdf,30-04-2023,9.5,222.0,"[B-74, Mehuwala Mafi, Dehradun, Uttarakhand 24..."
OLA_7.pdf,14-03-2018,46.8,1129.0,"[786, 1st Main Rd, 1st Block, Dubasi Palya, Ke..."
OLA_8.pdf,08-08-2017,,71.0,"[N1, Khirki Main Road, Savitri Nagar, Block M,..."
OLA_9.pdf,10-06-2022,,2251.0,"[Mangalore international Airport Mangalore, Ka..."
OLA_10.pdf,25-02-2019,,248.0,"[Gate no 1 Towards MG Rd, Iffico Chowk Metro S..."


In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import os
import multiprocessing
from gensim.models import Word2Vec
import re # For preprocessing
import pandas as pd # For data handling
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import SGDClassifier
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.remove('m')  #meter word must not be removed during preprocessing

In [26]:
def preprocess_text(text):
    #remove punctuation mark from the text
    text=text.translate(str.maketrans('','',''',!"#%&'()*+-/:;<=.>?@[\]^_`{|}~'''))
    # lower case each letter of the word
    text=text.lower()
    #including space in beween {num}{word} 
    text=re.sub(r'(\d+)([a-z]+)', r'\1 \2', text)
    # split the words into tokens
    tokens=word_tokenize(text)
    # remove stop words 
    tokens = [word for word in tokens if word not in stop_words]
    #all words to root words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    #lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    ret=[]
    for w in tokens:
        if w.isalpha():
           ret.append(w)
    return ret

def get_docs(folder):
    dir=folder+'_CSV'
    docs=[]
    for invoice_dir in os.listdir(dir):
        doc=[]
        for csv in os.listdir(dir+'/'+invoice_dir):
            df=pd.read_csv(dir+'/'+invoice_dir+'/'+csv)
            df=df.dropna()
            for i in df['text']:
                doc.extend(preprocess_text(i))
        docs.append(doc)        
    return docs
    

In [27]:
cores=multiprocessing.cpu_count()
w2v_model = Word2Vec(window=2,min_count=1,workers=cores-1)
docs=get_docs('OLA')
docs.extend(get_docs('UBER'))
docs.extend(get_docs('RAPIDO'))
w2v_model.build_vocab(docs)
# w2v_model.wv.index_to_key()
w2v_model.train(docs, total_examples=w2v_model.corpus_count, epochs=25)
# w2v_model.save('km_m_model.w2v')

(275884, 362100)

In [47]:
w2v_model.wv.most_similar('total')

[('subtot', 0.9802053570747375),
 ('toll', 0.9759376049041748),
 ('surcharg', 0.9712119102478027),
 ('promot', 0.9693785905838013),
 ('fare', 0.9549627900123596),
 ('addit', 0.9477730393409729),
 ('modifi', 0.9472338557243347),
 ('applic', 0.9454696774482727),
 ('cgst', 0.9453719258308411),
 ('payabl', 0.9449303150177002)]

In [48]:
w2v_model.save('embed.model')

In [49]:
ls

embed.wv            OLA_IMAGES/  RAPIDO/         train.ipynb  UBER_IMAGES/
extract_data.ipynb  ola.json     RAPIDO_CSV/     try.png      uber.json
OLA/                page.png     RAPIDO_IMAGES/  UBER/
OLA_CSV/            rapido/      rapido.json     UBER_CSV/


In [63]:
model=Word2Vec.load('embed.model')
model.wv.most_similar('bengaluru')

[('devanah', 0.945830225944519),
 ('whitefield', 0.9222968220710754),
 ('west', 0.9168739914894104),
 ('windmil', 0.914400577545166),
 ('bengal', 0.9108327627182007),
 ('domlur', 0.9095456600189209),
 ('amaniker', 0.9091464281082153),
 ('atc', 0.9069696068763733),
 ('balanagar', 0.9066340923309326),
 ('termin', 0.9059497117996216)]